# PROCESO ETL (Extracción, Transformación y Carga de Datos) - siniestros viales

En este proyecto, nos embarcamos en la tarea crucial de abordar y mitigar los siniestros viales que afectan a la ciudad de Buenos Aires, Argentina. Con el objetivo principal de reducir las tragedias en las vías urbanas, nos sumergiremos en el análisis de datos relacionados con incidentes viales.

Nuestra misión es transformar datos crudos en conocimientos significativos que permitan comprender a fondo los patrones y factores subyacentes que contribuyen a los accidentes de tráfico. A través de un proceso ETL (Extract, Transform, Load), daremos forma a conjuntos de datos dispersos para obtener información valiosa.

El propósito último es proporcionar a las autoridades, organizaciones de tráfico y ciudadanos en general una visión clara y detallada de las áreas de mayor riesgo, los momentos críticos y los elementos que más influyen en la seguridad vial. Con esta información, se pretende impulsar la implementación de medidas preventivas y estrategias eficaces que contribuyan a la reducción significativa de las tragedias viales en la Ciudad Autónoma de Buenos Aires.

### Importamos BIBLIOTECAS necesarias para nuestros primeros pasos

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### CARGA DE LOS DATOS

Creamos una __FUNCION__ para cargar los archivos.
Carga datos desde un archivo Excel y devuelve un diccionario de DataFrames.

    Parameters:
    - archivo (str): Ruta del archivo Excel.
    - hojas (list): Lista de nombres de hojas a cargar.
    - engine (str, optional): Motor de Excel a utilizar. Por defecto, 'openpyxl'.

    Returns:
    dfs: Un diccionario donde las claves son los nombres de las hojas y los valores son DataFrames correspondientes.

    Example:
    >>> datos = cargar_datos_desde_excel('archivo.xlsx', ['Hoja1', 'Hoja2'])
    >>> df_hoja1 = datos['Hoja1']
    >>> df_hoja2 = datos['Hoja2']


In [3]:
def cargar_datos_desde_excel(archivo, hojas, engine='openpyxl'):    
    xls_file = pd.ExcelFile(archivo, engine=engine)
    dfs = {}

    for hoja in hojas:
        df = pd.read_excel(xls_file, hoja) 
        dfs[hoja] = df

    return dfs

In [5]:
datos_homicidios = cargar_datos_desde_excel('homicidios.xlsx', ['HECHOS', 'VICTIMAS'])
datos_homicidios = cargar_datos_desde_excel('lesiones.xlsx', ['HECHOS', 'VICTIMAS'])

### EXPLORACION Y LIMPIEZA DE LOS DATOS

__DataSet:__ HOMICIOS - HECHOS

In [6]:
# Accedemos al dataframe por nombre de hoja, homicidios - hechos

h_h_df = datos_homicidios ['HECHOS']

In [9]:
#Visualizacion de las primeras filas del dataframe

print('DataFrame ORIGIANAL:')
h_h_df.head(5)

DataFrame ORIGIANAL:


,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,-34.647349,PEATON,SD,PEATON-SD,x,SD,SD,SD,SD,SD
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,-34.604579,AUTO,SD,AUTO-SD,SD,SD,x,SD,SD,SD


In [10]:
#Obtener informacion general del Dataframe

h_h_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

Tras verificar el diccionario de datos, identificamos que los valores "SD" corresponden a "Sin datos". Por consiguiente, tomamos la decisión de cambiar estos valores a NaN para estandarizar.

Creamos una __FUNCION__ para Analizar los valores.

Analiza la presencia de valores 'SD' en cada columna del DataFrame.

    Parameters:
    dataframe (pd.DataFrame): El DataFrame a analizar.

    Returns:
    pd.DataFrame: Un DataFrame que muestra la cantidad y porcentaje de valores 'SD' en cada columna.

In [11]:
def analizar_valores_sd(dataframe):

    columnas_con_sd = dataframe.columns
    resultados = []

    for columna in columnas_con_sd:
        cantidad_sd = dataframe[columna].eq('SD').sum()
        porcentaje_sd = (cantidad_sd / len(dataframe)) * 100
        resultados.append({'Columna': columna, 'Cantidad de SD': cantidad_sd, 'Porcentaje de SD': porcentaje_sd})

    resultados_df = pd.DataFrame(resultados)
    resultados_con_sd = resultados_df[resultados_df['Cantidad de SD'] > 0]

    return resultados_con_sd

In [13]:
# Llamamos la funcion analizar valores

resultados_h_hechos = analizar_valores_sd(h_h_df)
resultados_h_hechos

,Columna,Cantidad de SD,Porcentaje de SD
6,hora,4,0.016817
8,direccion_normalizada,10815,45.469834
9,comuna,846,3.556864
10,tipo_calle,11045,46.436830
11,otra_direccion,18295,76.918226
15,geocodificacion_CABA,1213,5.099853
16,longitud,1209,5.083036
17,latutid,1209,5.083036
18,victima,10733,45.125079
19,acusado,15288,64.275804


In [14]:
#Remplazar SD por NaN en todo el DataFrame

h_h_df.replace(['SD','sd'], np.nan, inplace=True)

Creamos una __FUNCION__ para comenzar con el proceso de limpieza de datos en un DataFrame.

    Parámetros:
    - df (pd.DataFrame): El DataFrame que se va a limpiar.
    
    - drop_duplicates (bool): Elimina duplicados si es True.
      Ejemplo: cleaned_df = data_cleaning(df_tu_data_frame, drop_duplicates=True)
    
    - drop_na (bool): Elimina filas con valores nulos si es True.
      Ejemplo: cleaned_df = data_cleaning(df_tu_data_frame, drop_na=True)

    - fill_na (dict): Un diccionario donde las claves son los nombres de columnas y los valores son valores para rellenar los nulos.
      Ejemplo: fill_na_dict = {'gravedad': 'leve'}
               cleaned_df = data_cleaning(df_tu_data_frame, fill_na=fill_na_dict)
        
    - convert_to_datetime (list): Lista de columnas para convertir a tipo de dato datetime.
      Ejemplo: columns_to_convert = ['fecha', 'hora']
               cleaned_df = data_cleaning(df_tu_data_frame, convert_to_datetime=columns_to_convert)

    - uppercase_columns (list): Lista de columnas para convertir a mayúsculas.
      Ejemplo: columns_to_uppercase = ['nombre', 'apellido']
               cleaned_df = data_cleaning(df_tu_data_frame, uppercase_columns=columns_to_uppercase)

    - lowercase_columns (list): Lista de columnas para convertir a minúsculas.
      Ejemplo: columns_to_lowercase = ['Ciudad', 'Pais']
               cleaned_df = data_cleaning(df_tu_data_frame, lowercase_columns=columns_to_lowercase)

    - titlecase_columns (list): Lista de columnas para convertir a formato de título (primera letra en mayúscula, resto en minúscula).
      Ejemplo: columns_to_titlecase = ['titulo', 'categoria']
               cleaned_df = data_cleaning(df_tu_data_frame, titlecase_columns=columns_to_titlecase)
            
    - strip_spaces (bool): Elimina espacios en blanco alrededor de los valores de las celdas si es True.
      Ejemplo: cleaned_df = data_cleaning(df_tu_data_frame, strip_spaces=True)

    - rename_columns (dict): Un diccionario donde las claves son los nombres de las columnas actuales y los valores son los nuevos nombres.
      Ejemplo: rename_dict = {'Vieja_Columna': 'Nueva_Columna'}
               cleaned_df = data_cleaning(df_tu_data_frame, rename_columns=rename_dict)
    
    - drop_columns (list): Lista de columnas para eliminar.
      Ejemplo: columns_to_drop = ['columna1', 'columna2']
               cleaned_df = data_cleaning(df_tu_data_frame, drop_columns=columns_to_drop)
        
    - categorize_columns (list): Lista de columnas para convertir a tipo de dato categoría.
      Ejemplo: columns_to_categorize = ['categoria1', 'categoria2']
               cleaned_df = data_cleaning(df_tu_data_frame, categorize_columns=columns_to_categorize)
        
    - replace_values (dict): Un diccionario donde las claves son los nombres de las columnas y los valores son diccionarios de reemplazo.
      Ejemplo: replace_dict = {'columna1': {'Antiguo1': 'Nuevo1', 'Antiguo2': 'Nuevo2'}}
               cleaned_df = data_cleaning(df_tu_data_frame, replace_values=replace_dict)

    - new_columns (dict): Un diccionario donde las claves son los nombres de las nuevas columnas y los valores son valores para esas columnas.
      Ejemplo: new_columns_dict = {'nueva_columna': 0}
               cleaned_df = data_cleaning(df_tu_data_frame, new_columns=new_columns_dict)
               
    - new_columns2 (dict): Un diccionario donde las claves son los nombres de las nuevas columnas y los valores son expresiones
                          para calcular el contenido de las nuevas columnas basadas en otras columnas existentes. 
      Ejemplo: {'nueva_columna1': 'columna_existente * 2'}
      cleaned_df = data_cleaning(df_tu_data_frame, new_columns2=new_columns_dict)

 
    - convert_date_columns (dict): Un diccionario donde las claves son los nombres de las columnas y los valores son los formatos de fecha.
      Ejemplo: date_columns_dict = {'fecha': '%Y-%m-%d', 'hora': '%H:%M:%S'}
               cleaned_df = data_cleaning(df_tu_data_frame, convert_date_columns=date_columns_dict)

    - convert_to_int_columns (list): Lista de columnas para convertir a tipo de dato entero.
      Ejemplo: columns_to_int = ['columna1', 'columna2']
               cleaned_df = data_cleaning(df_tu_data_frame, convert_to_int_columns=columns_to_int)
    
    - convert_to_float (list): Lista de columnas para convertir a tipo de dato float.
      Ejemplo: columns_to_float = ['columna1', 'columna2']
               cleaned_df = data_cleaning(df_tu_data_frame, convert_to_float=columns_to_float)          
            
    Retorna:
    pd.DataFrame: El DataFrame limpio.


In [15]:
def data_cleaning(df, drop_duplicates=False, drop_na=False, fill_na=None, convert_to_datetime=None, uppercase_columns=None,
                  lowercase_columns=None, titlecase_columns=None, strip_spaces=True, rename_columns=None, drop_columns=None,
                  categorize_columns=None, replace_values=None, new_columns=None, convert_date_columns=None, 
                  convert_to_int_columns=None, convert_to_float=None, new_columns2=None):

    cleaned_df = df.copy()

    # Eliminar duplicados
    if drop_duplicates:
        cleaned_df.drop_duplicates(inplace=True)
        

    # Eliminar filas con valores nulos
    if drop_na:
        cleaned_df.dropna(inplace=True)
        

    # Rellenar valores nulos
    if fill_na:
        cleaned_df.fillna(fill_na, inplace=True)

        
    # Convertir columnas a tipo datetime
    if convert_to_datetime:
        for column in convert_to_datetime:
            cleaned_df[column] = pd.to_datetime(cleaned_df[column], errors='coerce')
            

    # Convertir columnas a mayúsculas
    if uppercase_columns:
        for column in uppercase_columns:
            cleaned_df[column] = cleaned_df[column].str.upper()
            

    # Convertir columnas a minúsculas
    if lowercase_columns:
        for column in lowercase_columns:
            cleaned_df[column] = cleaned_df[column].str.lower()
        

    # Convertir columnas a formato de título
    if titlecase_columns:
        for column in titlecase_columns:
            cleaned_df[column] = cleaned_df[column].str.title()
            
            
    # Tratar columnas con espacios
    if strip_spaces:
        cleaned_df = cleaned_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        

    # Renombrar columnas
    if rename_columns:
        cleaned_df.rename(columns=rename_columns, inplace=True)
        
    
    # Eliminar columnas
    if drop_columns:
        cleaned_df.drop(columns=drop_columns, inplace=True)
        
        
    # Categorizar columnas
    if categorize_columns:
        for column in categorize_columns:
            if column in cleaned_df.columns:
                cleaned_df[column] = cleaned_df[column].astype('category')
            else:
                print(f"La columna '{column}' no existe en el DataFrame.")
                

    # Reemplazar valores en columnas
    if replace_values:
        for column, replacements in replace_values.items():
            cleaned_df[column].replace(replacements, inplace=True)
            
    # Agregar nuevas columnas
    if new_columns:
        for column, value in new_columns.items():
            cleaned_df[column] = value
            
    # Agregar nuevas columnas basadas en otras columnas
    if new_columns2:
        for new_column, column_expr in new_columns2.items():
            # Verificar si la expresión es proporcionada
            if column_expr:
                cleaned_df[new_column] = cleaned_df.eval(column_expr)
            else:
                cleaned_df[new_column] = None  # O cualquier valor predeterminado que prefieras
                  
    # Convertir columnas de fecha con formato específico
    if convert_date_columns:
        for column, date_format in convert_date_columns.items():
            cleaned_df[column] = pd.to_datetime(cleaned_df[column], format=date_format, errors='coerce')

    
    # Convertir columnas a tipo de dato entero
    if convert_to_int_columns:
        for column in convert_to_int_columns:
            cleaned_df[column] = pd.to_numeric(cleaned_df[column], errors='coerce').astype('Int64')
    
    
    # Convertir columnas a tipo float
    if convert_to_float:
        for column in convert_to_float:
            cleaned_df[column] = cleaned_df[column].astype(float)
        
            
    return cleaned_df

In [ ]:
# Iniciamos la preparación de los diccionarios y listas que alimentaran la función 'data_cleaning' 

In [16]:
# Columnas para pasar a minúcula

columns_to_lower = ['TIPO_DE_CALLE', 'VICTIMA', 'ACUSADO']

Convertir todos los datos a minúsculas evita posibles problemas de coincidencia y simplificas las operaciones de búsqueda y filtrado.

In [17]:
# Columnas para eliminar

columns_to_drop = ['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO','Calle','Altura',
                   'Cruce','Dirección Normalizada','XY (CABA)','PARTICIPANTES']

Decidimos eliminar las columnas 'AAAA', 'MM', y 'DD' del conjunto de datos, ya que la información de fecha ya está presente en la columna 'FECHA'. Esta decisión simplifica el conjunto de datos, mantiene una estructura más consistente y estandarizada, ahorra espacio y facilita el análisis temporal.

Asimismo, eliminamos la columna 'HORA' debido a la redundancia de la información de hora, que ya está contenida en la columna 'HH'. Esta elección nos proporciona una visión más general y simplificada de la distribución de los siniestros a lo largo del día.

Las columnas 'LUGAR_DEL_HECHO','Calle','Altura','Cruce' y 'Dirección Normalizada' implicaban datos de la dirección del hecho incluso, 'XY (CABA)' que contenía coordenadas en formato de proyección cartesiana también fue eliminada. Preferimos trabajar con las columnas de latitud y longitud por su mayor intuición, no tiene datos nulos, familiaridad para la mayoría de las personas, facilidad de representación gráfica en mapas y conformidad con estándares de sistemas de información geográfica (SIG).

La columna 'PARTICIPANTES' contenia información concatenada redundante de 'VICTIMA' y 'ACUSADO', así que se mantuvieron éstas dos últimas.

In [18]:
# Renombrar columnas

rename_dict = {'ID': 'id_siniestro', 'N_VICTIMAS': 'nro_victimas', 'FECHA': 'fecha', 'HH': 'franja_hora',
               'TIPO_DE_CALLE': 'tipo_calle', 'COMUNA': 'comuna', 'pos x': 'longitud', 'pos y': 'latitud',
               'VICTIMA': 'vehiculo_victima', 'ACUSADO': 'vehiculo_acusado'}

Renombramos columnas para mejorar la claridad y legibilidad del conjunto de datos.

In [19]:
# Convertir a entero

columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']